In [1]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
from scipy.stats import pearsonr
from numpy.typing import NDArray

2023-01-11 13:30:48.468156: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
class Scorer:
    
    def __init__(self, 
                 model_names: list[str], 
                 images: NDArray[np.float32], 
                 labels: NDArray[np.uint8]) -> None:
        self.model_names = model_names
        self.images, self.labels = images, labels
        # load models
        self.base_models = list()
        for m_name in self.model_names:
            self.base_models.append(tf.keras.models.load_model(m_name))
            
    def score_layer(self, 
                    layer_name: str, 
                    preds: list[NDArray[np.float32]]) -> list[float]:
        scores = list()
        for score in range(32):
            print(score, end='\r', flush=True)
            per_channel_corr = list()
            for prediction in preds[1:]:
                for channel in range(32):
                    first, second = list(), list()
                    for i in range(self.images.shape[0]):
                        # 0th model (the one that is being scored)
                        first.append(np.linalg.norm(preds[0][i,:,:,score]))
                        # 1..nth model
                        second.append(np.linalg.norm(prediction[i,:,:,channel]))
                    per_channel_corr.append(pearsonr(first, second)[0])
            scores.append(max(per_channel_corr))
        return scores

    def score_multiple_layers(self, 
                              layers: list[str]) -> dict[str, list[float]]:
        scores = dict()
        for layer_name in layers:
            # create intermediate models
            inter_models = list()
            for i in range(len(self.model_names)):
                input_layer = self.base_models[i].input
                output_layer = self.base_models[i].get_layer(layer_name).output
                inter_models.append(tf.keras.Model(inputs=input_layer, 
                                                   outputs=output_layer))
            # compute itermediate predictions
            preds = list()
            for i in range(len(self.model_names)):
                preds.append(inter_models[i].predict(test_images))
            scores[layer_name] = self.score_layer(layer_name, preds)
        return scores

In [3]:
model_names = list()
for i in range(5):
    name = 'cifar10/models/cifar_model_' + str(i) + '.h5'
    model_names.append(name)
_, (test_images, test_labels) = tf.keras.datasets.cifar10.load_data()
s = Scorer(model_names, 
           test_images, 
           test_labels)
x = s.score_multiple_layers(['conv2d', 'conv2d_1'])
# TODO: change channel count from 32 to [-1]

2023-01-11 12:52:40.880973: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-11 12:52:40.882733: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


313/313 [==============================] - 3s 9ms/step
